# Everything starts here

In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import utils

## 1. Split train / validation

In [2]:
# do the split only if it has not already been done
if not os.path.exists(utils.dfs_path + '/training.pkl') or not os.path.exists(utils.dfs_path + '/validation.pkl'):
    # load all train videos (labelled videos)
    all_train_videos = utils.get_train_test_video_names()['train']
    all_train_labels = pd.read_pickle(utils.labels_path)

    # define split
    split = 0.8
    np.random.seed(69)
    train_videos = np.array(all_train_videos)[np.random.choice(len(all_train_videos), int(0.8 * len(all_train_videos)), replace=False)]
    validation_videos = np.setdiff1d(all_train_videos, train_videos, assume_unique=False)
    train_videos.sort()
    validation_videos.sort()

    # create two subdataframes for training and validation
    training_df = all_train_labels.loc[all_train_labels['videoname'].isin(train_videos)]
    validation_df = all_train_labels.loc[all_train_labels['videoname'].isin(validation_videos)]

    training_df.to_pickle(utils.dfs_path + '/training.pkl')
    validation_df.to_pickle(utils.dfs_path + '/validation.pkl')

## 2. Model construction

In [3]:
# build model
model = models.mobilenet_v2(pretrained=True).to(utils.device)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, utils.num_classes, device = utils.device)
model_name = 'mobilenet_v2'

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

2241806

## 3. Set training hyperparameters

In [4]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
# training parameters
LEARNING_RATE = 0.005
EPOCHS = 3
BATCH_SIZE = 128
MOMENTUM = 0.9
GAMMA = 0.3
STEP_SIZE = 1

In [6]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [7]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [8]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['training', 'validation']}

## 4. Train

In [9]:
utils.train_model(model = model, 
                    model_name = model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/3
----------
164841/164841: [===============================>] - ETA 3.0ssss
training Loss: 0.3677 Acc: 0.8753
50216/50216: [===============================>] - ETA 0.9sss
validation Loss: 0.7777 Acc: 0.7949
Epoch 2/3
----------
164841/164841: [===============================>] - ETA 0.8sss
training Loss: 0.1459 Acc: 0.9511
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.7477 Acc: 0.8026
Epoch 3/3
----------
164841/164841: [===============================>] - ETA 0.8ssss
training Loss: 0.1042 Acc: 0.9654
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.7721 Acc: 0.8015
Training complete in 70m 29s
Best val Acc: 0.802593


## 5. Make Kaggle prediction

In [2]:
# create and save testing df
utils.save_testing_df()

'testing dataframe already in storage'

In [3]:
# rebuild model
model = models.mobilenet_v2(pretrained=True).to(utils.device)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, utils.num_classes, device = utils.device)
model_name = 'mobilenet_v2'

# data augmentation and normalization for training
testing_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [4]:
utils.predict_kaggle(model = model, 
                    model_name = model_name, # name of the model from which to load the weights within weights/
                    transform = testing_transforms, 
                    predictions_name = model_name,
                    batch_size = 64) # name of the csv file to which the predictions are saved within predictions/

184003/184003: [===============================>] - ETA 0.7ssss
